# Корреляция

In [1]:
# раскомментирвоть (убрать символ #) строки ниже для установки модулей в mybinder.org
# !pip install scipy
# !pip install seaborn
# !pip install pandas

In [ ]:
import pandas as pd     # для загрузки и хранения данных в таблице (DataFrame)
import seaborn          # для построения графиков
from matplotlib.pyplot import *   # для построения графиков

from scipy.stats import linregress, pearsonr    # для построения уравнения регрессии
import numpy as np

## Пример 1
Поглощение звука материалами в зависимости от их свойств (толщина, масса, жесткость и др.)

Источник данных: 
http://users.stat.ufl.edu/~winner/datasets.html


Описание данных:

Description: Acoustic absorption coefficients of 24 woven fabrics
with different air gap distances (d=0,1,2,3,cm)

Responses: 

- acoustic0  (d=0 cm)
- acoustic1
- acoustic2
- acoustic3


Predictors:

- thickness   (mm)
- diameter    (mm)
- perforation (%)
- weight      (g/m^2)
- stiffness   (mN x cm)
- airPerm     (air permeability, mm/s) 

In [ ]:
# загрузка данных из файла fabricsoundabsorb.csv 
# это текстовый файл с данными расположеными в столбцах
# header = 0 - заголовок для столбцов это нелувая (персвая посчёту строка)
# comment='#' решеткой в начале строки помечаны дополнительная, справочная, информация
# в файле, они будут проигнорированы при загрузке.

data = pd.read_csv("fabricsoundabsorb.csv", comment='#', header=0)

# данные будут загружены в переменную data

In [ ]:
# В mybinder.org файлы могут быть недоступны,
# тогда записать данные в переменную data можно скопировав их из файла.


# все данные записываются в квадратных скобках, каждая строчка из файла снова в квадратных скобках
# нужно раскомментировать строки ниже (убрать символ #)
# d = [ [1213, 21241, 31412, 41242],
#       [623,  14324, 231,   12123]]
# 
# data = pd.DataFrame( d )

In [ ]:
# напечатаем эти данные
data

Загруженные данные -- это таблица.

Столбец  sampleID -- это номер образца материала. Для вычисления КК это число бесполезно, поэтому удалим столбец из таблицы.



In [ ]:
data = data.drop(columns=['sampleID'])

In [ ]:
# завново напечатем данные, но только первые 10 строк
data.head()

### Корреляционный анализ

**Корреляционный анализ** — метод обработки статистических данных, с помощью которого измеряется теснота связи между двумя или более переменными.

**Корреляционный анализ** — метод обработки статистических данных, заключающийся в изучении коэффициентов корреляции между переменными. 

Вычислим и выведем коэффициенты корреляции для каждого столбца таблицы с каждым.
Нижеприведенная таблица называется **матрицей корреляции**.

In [ ]:
data.corr()

Рассмотрим например корреляцию между коэф. поглощения звука (acoustic1) и толщиной материала (thickness): пересечение строки thickness и столбца acoustic1, КК  0.613085.

- Коэффициент корреляции отличается от нуля, значит существует зависимость между поглощением звука и толщиной материала. 
- КК положителен, значит эта зависимость прямая
- Далее будем обозначать КК буквой **r**.


Аналогичным способом можно делать и выводы о кореляции других величин друг от друга.

Однако можно представить матрицу корреляции и нагляднее. Построим тепловую карту, обозначив величину КК цветом.

In [ ]:
figure( figsize=(10,10))  # размер графика - 10х10 едениц
seaborn.heatmap( data.corr(),
                cmap='coolwarm',  # способ цветового кодирования: холодные и тёплые цвета
                annot=True)  # annot=True - показывать не только цвета, но и величну КК

Теперь рассмотрим корреляцию между толщиной материала и коэф. звукопоглощения отдельно.

Построим диаграмму рассеивания. Точки на этой диаграмме - это отдельные измерения соответствующих величин.

In [ ]:
seaborn.jointplot(data=data,  # используемые данные
                  x="thickness", y='acoustic1', # какой столбец по какой оси озобразить
                  kind='reg')  # reg - изобразить линию регрессии

- Точки на графики - это пары значений (thickness и acoustic1)
- Каздая пара значений взята из одной строчки таблицы data. Одна строка таблицы - это одно испытание на звукопоглощение.
- На графике видно, что большинство значений грубо подходят под проведённую линию. 
- Эта линия называется **линией регрессии**
- Чем ближе КК к 1 или -1, тем больше точек лежит на линии регрессии

#### Гипотеза о равенстве КК нулю

- Однако, экспериментальные данные - это всего лишь выборка, потому, что исследуется только небольшая часть из всех материалов. Поэтому есть шанс, что полученная зависимость между исследуемыми величинами случайна.

- Требуется убедится, что зависимость есть не только в выборке но и в генеральной совокупности.

- При построении графика заодно проверяется гипотеза (нулевая гипотеза) о равенстве КК нулю. 
    - $H_0: r = 0$

- Если это равенство подтвердится, значит полученное значение КК получилось именно таким случайно. 

- Для проверки гипотезы вычисляется p-value. Правило для принятия нулевой гипотезы: 
    - p-value > 0.05 - нулевая гипотеза принимается.
    - p-value < 0.05 - нулевая гипотеза отвергается.



В примере выше: p-value = 0.0014 < 0.05.
Отвергаем нулевую гипотезу о равенстве КК нулю. КК статистически значим. Поэтому будем считать, что такое же значение КК справедливо и для выборки.


Просмотрим справку по функции, вычисляющей коэффицент корреляции и p-value

In [ ]:
help(pearsonr)

In [ ]:
# вычислим коэффициент корреляции и p-value
pearsonr( data["thickness"], data["acoustic1"])

В примере выше: p-value = 0.0014 < 0.05.
Отвергаем нулевую гипотезу о равенстве КК нулю. КК статистически значим. Поэтому будм считать, что такое же значение КК справделиво и для выборки.


***
Для примера построим ещё и диаграмму рассеивания для величин поглощение звука (acoustic1) и жёсткость (stiffness)

In [ ]:
seaborn.jointplot(data=data, x="stiffness", y="acoustic1", kind='reg')

- Здесь точки разбросаны более хаотично, трудно провести линию регрессии, которая подходила бы большинству точек.
- Поэтому и КК близок к нулю.
- Значит здесь нет заметной статистической зависимости
- Это подтверждается проверкой гипотезы о равенстве КК нулю: p-value > 0.05. Значит нет оснований отклонить нулевую гипотезу $H_0: r=0$

## Регрессионный анализ
**Регрессионный анализ** — статистический метод исследования влияния одной или нескольких независимых переменных $X_1 , X_2 , ... , X_n$ на зависимую переменную Y.


- найти функцию, которая отображает зависимость одних переменных или данных от других
    - Y = f(x_1, x_2, ..., x_3)
- функция строится на основе известных значений переменных
- Регрессия полезна для прогнозирования ответа на новые условия





Отличия регрессионного анализа от корреляционного:
- в корреляционном анализе важен коэффициент корреляции
- в регрессионном анализе важно уравнение регрессии, т.е. предсказание значений зависимой переменной

In [ ]:
linregress(data['airPerm'], data['acoustic1'])

Уравнение будет иметь вид:
$$ acoustic1 = slope \cdot airPerm + intercept $$


Теперь им можно пользоватся, чтобы предсказать *среднее* значение *acoustic1*, для заданного значения *airPerm* (и при фиксированых значениях остальных переменных)

In [ ]:
# например вычислим коэф. звукопоглощения1 (acoustic1)
# для воздухопроницаемости airPerm 950 mm/s
-0.0005196861386681389 * 950 + 0.7918330758205965

In [ ]:
# построим диаграмму рассеивания и линию регрессии
seaborn.jointplot(data['airPerm'], data['acoustic1'], kind='reg')

# отметим предсказанное значение коэф. звукопоглощения
plot(950, 0.2981312440858645, 'o')
grid()

# Пример 2

Concrete Compressive Strength 

---------------------------------

Data Type: multivariate
 
Abstract: Concrete is the most important material in civil engineering. The 
concrete compressive strength is a highly nonlinear function of age and 
ingredients. These ingredients include cement, blast furnace slag, fly ash, 
water, superplasticizer, coarse aggregate, and fine aggregate.

---------------------------------

Sources: 

  Original Owner and Donor
  Prof. I-Cheng Yeh
  Department of Information Management 
  Chung-Hua University, 
  Hsin Chu, Taiwan 30067, R.O.C.
  e-mail:icyeh@chu.edu.tw
  TEL:886-3-5186511

  Date Donated: August 3, 2007
 
---------------------------------

Data Characteristics:
    
The actual concrete compressive strength (MPa) for a given mixture under a 
specific age (days) was determined from laboratory. Data is in raw form (not scaled). 

Summary Statistics: 

Number of instances (observations): 1030
Number of Attributes: 9
Attribute breakdown: 8 quantitative input variables, and 1 quantitative output variable
Missing Attribute Values: None

---------------------------------

Variable Information:

Given is the variable name, variable type, the measurement unit and a brief description. 
The concrete compressive strength is the regression problem. The order of this listing 
corresponds to the order of numerals along the rows of the database. 

Name -- Data Type -- Measurement -- Description

- Cement (component 1) -- quantitative -- kg in a m3 mixture -- Input Variable
- Blast Furnace Slag (доменный шлак, component 2) -- quantitative -- kg in a m3 mixture -- Input Variable
- Fly Ash (Летучая зола, component 3) -- quantitative -- kg in a m3 mixture -- Input Variable
- Water (вода, component 4) -- quantitative -- kg in a m3 mixture -- Input Variable
- Superplasticizer (component 5) -- quantitative -- kg in a m3 mixture -- Input Variable
- Coarse Aggregate (заполнитель, component 6) -- quantitative -- kg in a m3 mixture -- Input Variable
- Fine Aggregate (мелкий заполнитель, component 7) -- quantitative -- kg in a m3 mixture -- Input Variable
- Age (время отведённое на завтердевание) -- quantitative -- Day (1~365) -- Input Variable
- Concrete compressive strength (прочность на сжатие) -- quantitative -- MPa -- Output Variable 

---------------------------------

Acknowledgements, Copyright Information, and Availability:

NOTE: Reuse of this database is unlimited with retention of copyright notice for 
Prof. I-Cheng Yeh and the following published paper:

I-Cheng Yeh, "Modeling of strength of high performance concrete using artificial 
neural networks," Cement and Concrete Research, Vol. 28, No. 12, pp. 1797-1808 (1998)





In [ ]:
data2 = pd.read_csv("Concrete_Data.csv", header=0, comment='#', delimiter=';')
# здесь один столбец отделён от другого точкой с запятой (;), а не запятой, 
# поэтому чтобы данные загрузились корректно, укажем разделитель: delimiter=';'

In [ ]:
# названия столбцов очень длинные, поэтому заменим на короткие
cols = data2.columns.values
cols[0] = "component1"
cols[1] = "component2"
cols[2] = "component3"
cols[3] = "component4"
cols[4] = "component5"
cols[5] = "component6"
cols[6] = "component7"
cols[7] = "age"
cols[8] = "strength"
data2.columns = cols

In [ ]:
# удалим последний столбец, он без данных
data2 = data2.drop(columns=['Unnamed: 9'])

In [ ]:
data2.head()

In [ ]:
data2.info()

In [ ]:
figure( figsize=(10,10) )
seaborn.heatmap( data2.corr(), cmap='coolwarm', annot = True )

- Какие части бетонной смеси сильнее всего влияет на его прочность после застывания?
- Где зависимость прямая, а где обратная?
- Есть ли компоненты смеси, которые мало влияют на прочность?
- Как влияет время застывания смеси на прочность бетона?
- Как влияет количество воды в смеси на время застывания?